<a href="https://colab.research.google.com/github/nathanguevara11/Home_Sales/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [91]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [92]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

homes_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"))

cols = ["id", "date", "date_built", "price", "bedrooms", "bathrooms", "sqft_living", "sqft_lot", "floors", "waterfront", "view"]

homes_df = homes_df.toDF(*cols)

row_one = homes_df.limit(1)

homes_df = homes_df.subtract(row_one)
homes_df.show()

+--------------------+----------+----------+-------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+-------+--------+---------+-----------+--------+------+----------+----+
|b672c137-b88c-48b...|2019-10-16|      2016| 239895|       2|        2|       1631|   11149|     2|         0|   0|
|08784e80-3b51-483...|2020-04-28|      2016| 273544|       2|        3|       1664|   10137|     2|         0|  22|
|ce519c6e-701a-4a2...|2022-03-05|      2015| 236997|       3|        3|       2487|   11245|     2|         0|  21|
|2da711ea-9ea0-456...|2021-09-28|      2017| 353653|       4|        2|       1557|   14907|     2|         0|  35|
|deb4e689-be36-47f...|2019-08-24|      2010| 265975|       3|        2|       1792|   13389|     2|         0|   9|
|baffb494-ff65-43e...|2021-09-24|      2010|1277874|       6|        4| 

In [93]:
# 2. Create a temporary view of the DataFrame.
homes_df.createOrReplaceTempView("home_sales")

In [94]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

spark.sql("""SELECT YEAR(date), Round(AVG(price), 2) FROM home_sales WHERE bedrooms = 4 GROUP BY YEAR(date)""").show()

+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           296363.88|
|      2019|            300263.7|
|      2020|           298353.78|
|      2021|           301819.44|
+----------+--------------------+



In [95]:
spark.sql("""SELECT date FROM home_sales """).show()

+----------+
|      date|
+----------+
|2019-10-16|
|2020-04-28|
|2022-03-05|
|2021-09-28|
|2019-08-24|
|2021-09-24|
|2021-06-12|
|2019-09-05|
|2021-03-30|
|2022-05-07|
|2019-09-12|
|2019-06-20|
|2020-02-16|
|2021-09-07|
|2022-01-04|
|2020-01-12|
|2020-12-29|
|2019-08-22|
|2019-06-05|
|2021-05-29|
+----------+
only showing top 20 rows



In [96]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?


spark.sql("""SELECT YEAR(date), Round(AVG(price), 2) FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY YEAR(date) ORDER BY YEAR(date) DESC""").show()

+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           292725.69|
|      2021|           294211.46|
|      2020|           294204.16|
|      2019|           287287.82|
+----------+--------------------+



In [97]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

spark.sql("""SELECT YEAR(date), Round(AVG(price), 2) FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY YEAR(date) ORDER BY YEAR(date) DESC""").show()

+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           290242.99|
|      2021|           296330.96|
|      2020|           292289.09|
|      2019|           289859.14|
+----------+--------------------+



In [98]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

spark.sql("""SELECT view, Round(AVG(price), 2) FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC""").show()

start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|   9|           401393.34|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
+----+--------------------+
only showing top 20 rows

--- 0.00010442733764648438 seconds ---


In [99]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [100]:
# 8. Check if the table is cached.
print(spark.catalog.isCached('home_sales'))

True


In [101]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.


spark.sql("""SELECT view, Round(AVG(price), 2) FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC""").show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|   9|           401393.34|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
+----+--------------------+
only showing top 20 rows

--- 0.00011301040649414062 seconds ---


In [102]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homes_df.write.partitionBy("date_built").parquet('parquet_descriptions', mode = "overwrite")

In [103]:
# 11. Read the parquet formatted data.
homes_df_2=spark.read.parquet('parquet_descriptions')

In [104]:
# 12. Create a temporary table for the parquet data
homes_df_2.createOrReplaceTempView("parquet_table")

In [105]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

spark.sql("""SELECT view, Round(AVG(price), 2) FROM parquet_table WHERE price >= 350000 GROUP BY view ORDER BY view DESC""").show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|   9|           401393.34|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
+----+--------------------+
only showing top 20 rows

--- 0.00012135505676269531 seconds ---


In [106]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [107]:
# 15. Check if the home_sales is no longer cached

if spark.catalog.isCached('home_sales'):
  print("STILL HERE")
else:
  print("ALL GOOD")


ALL GOOD
